In [ ]:
import os
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy import interpolate
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import confusion_matrix, cohen_kappa_score, f1_score
from tensorflow.keras.layers import (
    Conv1D, MaxPooling1D, GlobalAveragePooling1D,
    Dense, Activation, Flatten, Input, Multiply, Add, 
    Concatenate, BatchNormalization, Reshape, Dropout
)
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers, metrics, losses, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random

# ============================== Data Loading & Preprocessing ==============================
class BCICompetitionData:
    def __init__(self, base_path, task='MI'):
        self.base_path = base_path
        self.task = task
        self.samples_per_trial_original = 2250
        self.downsample_length = 1050
        self.channels = ['C3', 'CZ', 'C4']
        
    def load_data(self, df):
        data_list = []
        label_list = []
        for idx, row in df.iterrows():
            trial_data = self.load_trial_data(row)
            eeg_data = trial_data[self.channels].values
            
            # Downsample and normalize
            eeg_data_down = self.interpolate_data(eeg_data, self.samples_per_trial_original, self.downsample_length)
            eeg_data_down = (eeg_data_down - eeg_data_down.mean(axis=0)) / (eeg_data_down.std(axis=0) + 1e-8)
            
            data_list.append(eeg_data_down)
            if 'label' in row:
                label_list.append(0 if row['label'] == 'Left' else 1)
                
        data_array = np.array(data_list)
        label_array = np.array(label_list)
        
        # One-hot encode labels
        if len(label_array) > 0:
            onehot_encoder = OneHotEncoder(sparse=False)
            label_array = onehot_encoder.fit_transform(label_array.reshape(-1, 1))
        return data_array, label_array
    
    def interpolate_data(self, data, orig, prolong, kind='linear'):
        new_data = []
        x_orig = np.linspace(0, prolong, orig)
        x_new = np.linspace(0, prolong, prolong)
        for i in range(data.shape[1]):
            f = interpolate.interp1d(x_orig, data[:, i], kind=kind)
            new_data.append(f(x_new))
        return np.array(new_data).T
    
    def load_trial_data(self, row):
        id_num = row['id']
        dataset = 'train' if id_num <= 4800 else 'validation' if id_num <= 4900 else 'test'
        
        eeg_path = os.path.join(
            self.base_path, 
            row['task'], 
            dataset, 
            row['subject_id'], 
            str(row['trial_session']), 
            'EEGdata.csv'
        )
        eeg_data = pd.read_csv(eeg_path)
        
        # Extract trial data
        trial_num = int(row['trial'])
        start_idx = (trial_num - 1) * 2250
        return eeg_data.iloc[start_idx:start_idx + 2250]
    
    def data_augmentation(self, data, label, windows_long, interval):
        new_data, new_labels = [], []
        for i in range(len(data)):
            trial = data[i]
            num_windows = (trial.shape[0] - windows_long) // interval + 1
            for j in range(num_windows):
                start = j * interval
                window = trial[start:start + windows_long]
                new_data.append(window)
                new_labels.append(label[i])
        return np.array(new_data), np.array(new_labels)
    
    def gauss_data_augmentation(self, data, label, sigma, m=2):
        if m == 1:
            return data, label
        new_data = [data]
        new_labels = [label]
        for _ in range(1, m):
            noisy_data = data + np.random.normal(0, sigma, data.shape)
            new_data.append(noisy_data)
            new_labels.append(label)
        return np.vstack(new_data), np.vstack(new_labels)

# ============================== Model Architecture ==============================
class MMCNN_model:
    def __init__(self, channels=3, samples=1000):
        self.channels = channels
        self.samples = samples
        self.activation = 'elu'
        self.learning_rate = 0.0001
        self.dropout = 0.8
        self.inception_filters = [16, 16, 16, 16]
        self.inception_kernel_length = [
            [5, 10, 15, 10],
            [40, 45, 50, 100],
            [60, 65, 70, 100],
            [80, 85, 90, 100],
            [160, 180, 200, 180],
        ]
        self.inception_stride = [2, 4, 4, 4, 16]
        self.first_maxpooling_size = 4
        self.first_maxpooling_stride = 4
        self.res_block_filters = [16, 16, 16]
        self.res_block_kernel_stride = [8, 7, 7, 7, 6]
        self.se_block_kernel_stride = 16
        self.se_ratio = 8
        self.second_maxpooling_size = [4, 3, 3, 3, 2]
        self.second_maxpooling_stride = [4, 3, 3, 3, 2]
        self.model = self.build_model()
        
        adam = optimizers.Adam(learning_rate=self.learning_rate) 
        self.model.compile(
            loss=losses.binary_crossentropy,
            optimizer=adam,
            metrics=['mae', metrics.binary_accuracy]
        )
    
    def build_model(self):
        input_tensor = Input(shape=(self.samples, self.channels))
        output_conns = []
        
        # EIN-a (Branch 0)
        x = self.inception_block(input_tensor, self.inception_filters, self.inception_kernel_length[0], self.inception_stride[0], self.activation)
        x = MaxPooling1D(pool_size=self.first_maxpooling_size, strides=self.first_maxpooling_stride, padding='same')(x)
        x = BatchNormalization()(x)
        x = Dropout(self.dropout)(x)
        x = self.conv_block(x, self.res_block_filters, self.res_block_kernel_stride[0], activation=self.activation)
        x = self.squeeze_excitation_layer(x, self.se_block_kernel_stride, self.activation, ratio=self.se_ratio)
        x = MaxPooling1D(pool_size=self.second_maxpooling_size[0], strides=self.second_maxpooling_stride[0], padding='same')(x)
        x = Flatten()(x)
        output_conns.append(x)
        
        # EIN-b (Branch 1)
        y1 = self.inception_block(input_tensor, self.inception_filters, self.inception_kernel_length[1], self.inception_stride[1], self.activation)
        y1 = MaxPooling1D(pool_size=self.first_maxpooling_size, strides=self.first_maxpooling_stride, padding='same')(y1)
        y1 = BatchNormalization()(y1)
        y1 = Dropout(self.dropout)(y1)
        y1 = self.conv_block(y1, self.res_block_filters, self.res_block_kernel_stride[1], activation=self.activation)
        y1 = self.squeeze_excitation_layer(y1, self.se_block_kernel_stride, self.activation, ratio=self.se_ratio)
        y1 = MaxPooling1D(pool_size=self.second_maxpooling_size[1], strides=self.second_maxpooling_stride[1], padding='same')(y1)
        y1 = Flatten()(y1)
        output_conns.append(y1)
        
        # EIN-c (Branch 2)
        y2 = self.inception_block(input_tensor, self.inception_filters, self.inception_kernel_length[2], self.inception_stride[2], self.activation)
        y2 = MaxPooling1D(pool_size=self.first_maxpooling_size, strides=self.first_maxpooling_stride, padding='same')(y2)
        y2 = BatchNormalization()(y2)
        y2 = Dropout(self.dropout)(y2)
        y2 = self.conv_block(y2, self.res_block_filters, self.res_block_kernel_stride[2], activation=self.activation)
        y2 = self.squeeze_excitation_layer(y2, self.se_block_kernel_stride, self.activation, ratio=self.se_ratio)
        y2 = MaxPooling1D(pool_size=self.second_maxpooling_size[2], strides=self.second_maxpooling_stride[2], padding='same')(y2)
        y2 = Flatten()(y2)
        output_conns.append(y2)
        
        # EIN-d (Branch 3)
        y3 = self.inception_block(input_tensor, self.inception_filters, self.inception_kernel_length[3], self.inception_stride[3], self.activation)
        y3 = MaxPooling1D(pool_size=self.first_maxpooling_size, strides=self.first_maxpooling_stride, padding='same')(y3)
        y3 = BatchNormalization()(y3)
        y3 = Dropout(self.dropout)(y3)
        y3 = self.conv_block(y3, self.res_block_filters, self.res_block_kernel_stride[3], activation=self.activation)
        y3 = self.squeeze_excitation_layer(y3, self.se_block_kernel_stride, self.activation, ratio=self.se_ratio)
        y3 = MaxPooling1D(pool_size=self.second_maxpooling_size[3], strides=self.second_maxpooling_stride[3], padding='same')(y3)
        y3 = Flatten()(y3)
        output_conns.append(y3)
        
        # EIN-e (Branch 4)
        z = self.inception_block(input_tensor, self.inception_filters, self.inception_kernel_length[4], self.inception_stride[4], self.activation)
        z = MaxPooling1D(pool_size=self.first_maxpooling_size, strides=self.first_maxpooling_stride, padding='same')(z)
        z = BatchNormalization()(z)
        z = Dropout(self.dropout)(z)
        z = self.conv_block(z, self.res_block_filters, self.res_block_kernel_stride[4], activation=self.activation)
        z = self.squeeze_excitation_layer(z, self.se_block_kernel_stride, self.activation, ratio=self.se_ratio)
        z = MaxPooling1D(pool_size=self.second_maxpooling_size[4], strides=self.second_maxpooling_stride[4], padding='same')(z)
        z = Flatten()(z)
        output_conns.append(z)
        
        # Combine all branches
        output_conns = Concatenate(axis=-1)(output_conns)
        output_conns = Dropout(self.dropout)(output_conns)
        output_tensor = Dense(2, activation='sigmoid')(output_conns)
        return Model(input_tensor, output_tensor)

    def squeeze_excitation_layer(self, x, out_dim, activation, ratio=8):
        squeeze = GlobalAveragePooling1D()(x)

        excitation = Dense(units=out_dim//ratio)(squeeze)
        excitation = Activation(activation)(excitation)
        excitation = Dense(units=out_dim, activation='sigmoid')(excitation)
        excitation = Reshape((1, out_dim))(excitation)

        scale = Multiply()([x, excitation])
        return scale
        
    '''
    res_block
    '''
    def conv_block(self, x, nb_filter, length, activation):
        k1, k2, k3 = nb_filter

        out = Conv1D(k1, length, strides=1, padding='same', 
                    kernel_regularizer=regularizers.l2(0.002))(x)
        out = BatchNormalization()(out)
        out = Activation(activation)(out)

        out = Conv1D(k2, length, strides=1, padding='same', 
                    kernel_regularizer=regularizers.l2(0.002))(out)
        out = BatchNormalization()(out)
        out = Activation(activation)(out)

        out = Conv1D(k3, length, strides=1, padding='same', 
                    kernel_regularizer=regularizers.l2(0.002))(out)
        out = BatchNormalization()(out)

        x = Conv1D(k3, 1, strides=1, padding='same')(x)
        x = BatchNormalization()(x)

        out = Add()([out, x])
        out = Activation(activation)(out)
        out = tf.keras.layers.Dropout(self.dropout)(out)
        return out
    
    '''
    inception_block 
    '''
    def inception_block(self, x, ince_filter, ince_length, stride, activation):
        k1, k2, k3, k4 = ince_filter
        l1, l2, l3, l4 = ince_length
        inception = []

        x1 = Conv1D(k1, l1, strides=stride, padding='same', 
                   kernel_regularizer=regularizers.l2(0.01))(x)
        x1 = BatchNormalization()(x1)
        x1 = Activation(activation)(x1)
        inception.append(x1)

        x2 = Conv1D(k2, l2, strides=stride, padding='same', 
                   kernel_regularizer=regularizers.l2(0.01))(x)
        x2 = BatchNormalization()(x2)
        x2 = Activation(activation)(x2)
        inception.append(x2)

        x3 = Conv1D(k3, l3, strides=stride, padding='same', 
                   kernel_regularizer=regularizers.l2(0.01))(x)
        x3 = BatchNormalization()(x3)
        x3 = Activation(activation)(x3)
        inception.append(x3)

        x4 = MaxPooling1D(pool_size=l4, strides=stride, padding='same')(x)
        x4 = Conv1D(k4, 1, strides=1, padding='same')(x4)
        x4 = BatchNormalization()(x4)
        x4 = Activation(activation)(x4)
        inception.append(x4)
        
        v1 = Concatenate(axis=-1)(inception)
        return v1
    
    # ... [Helper methods: inception_block, conv_block, squeeze_excitation_layer] ...

# ============================== Evaluation Metrics ==============================
class Evaluations:
    def __init__(self, history, y_pred, y_true, loss_score, error, validation_score):
        self.history = history
        self.y_pred = y_pred
        self.y_true = y_true
        self.loss_score = loss_score
        self.error = error
        self.validation_score = validation_score
        self.matrix, self.kappa = self.matrix_and_kappa()
        self.f1 = self.compute_f1_score()
    
    def matrix_and_kappa(self):
        y_pred_labels = np.argmax(self.y_pred, axis=1)
        y_true_labels = np.argmax(self.y_true, axis=1)
        C2 = confusion_matrix(y_true_labels, y_pred_labels)
        kappa_value = cohen_kappa_score(y_true_labels, y_pred_labels)
        return C2, kappa_value
    
    def compute_f1_score(self):
        y_pred_labels = np.argmax(self.y_pred, axis=1)
        y_true_labels = np.argmax(self.y_true, axis=1)
        return f1_score(y_true_labels, y_pred_labels, average='weighted')

# ============================== Main Training Pipeline ==============================
def main():
    # Configuration
    base_path = '/kaggle/input/mtcaic3'
    task = 'MI'
    
    # Load metadata
    train_df = pd.read_csv(f'{base_path}/train.csv')
    val_df = pd.read_csv(f'{base_path}/validation.csv')
    test_df = pd.read_csv(f'{base_path}/test.csv')
    
    # Filter for MI task
    train_df = train_df[train_df['task'] == task]
    val_df = val_df[val_df['task'] == task]
    
    # Initialize data loader
    data_loader = BCICompetitionData(base_path, task)
    
    # Load and preprocess data
    train_data, train_labels = data_loader.load_data(train_df)
    val_data, val_labels = data_loader.load_data(val_df)
    
    # Data augmentation parameters
    window_long = 1000
    window_val_interval = 10
    window_test_interval = 50
    
    # Augment training data
    train_data_aug, train_labels_aug = data_loader.data_augmentation(
        train_data, train_labels, window_long, window_val_interval
    )
    train_data_aug, train_labels_aug = data_loader.gauss_data_augmentation(
        train_data_aug, train_labels_aug, 0.005, m=2
    )
    
    # Augment validation data
    val_data_aug, val_labels_aug = data_loader.data_augmentation(
        val_data, val_labels, window_long, window_test_interval
    )
    
    # Initialize and train model
    model = MMCNN_model(channels=3, samples=1000).model
    
    checkpoint = ModelCheckpoint(
        'best_model.h5',
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    )
    early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
    
    history = model.fit(
        train_data_aug, train_labels_aug,
        validation_data=(val_data_aug, val_labels_aug),
        epochs=100,
        batch_size=128,
        callbacks=[checkpoint, early_stop],
        verbose=1
    )
    
    # Evaluate on validation set
    val_loss, val_mae, val_acc = model.evaluate(val_data_aug, val_labels_aug, verbose=0)
    y_pred = model.predict(val_data_aug)
    eval_results = Evaluations(history, y_pred, val_labels_aug, val_loss, val_mae, val_acc)
    
    print(f"Validation Accuracy: {val_acc:.4f}")
    print(f"F1 Score: {eval_results.f1:.4f}")
    print(f"Cohen's Kappa: {eval_results.kappa:.4f}")
    
    # Generate test predictions
    test_data, _ = data_loader.load_data(test_df)
    test_data_aug, _ = data_loader.data_augmentation(
        test_data, np.zeros((len(test_data), 2)), window_long, window_test_interval
    )
    test_probs = model.predict(test_data_aug)
    
    # Aggregate predictions per trial
    final_preds = []
    window_size = test_data_aug.shape[0] // len(test_data)
    for i in range(len(test_data)):
        trial_probs = test_probs[i * window_size : (i + 1) * window_size]
        avg_probs = np.mean(trial_probs, axis=0)
        final_preds.append('Left' if np.argmax(avg_probs) == 0 else 'Right')
    
    # Create submission file
    submission_df = pd.read_csv(f'{base_path}/sample_submission.csv')
    test_df_full = pd.read_csv(f'{base_path}/test.csv')
    
    # Fill predictions only for MI trials
    mi_mask = test_df_full['task'] == 'MI'
    submission_df.loc[mi_mask, 'label'] = final_preds
    submission_df.to_csv('submission.csv', index=False)
    print("Submission file created!")

if __name__ == "__main__":
    main()

2025-06-23 08:53:58.410655: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750668838.653627      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750668838.723599      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be remov

Epoch 1/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step - binary_accuracy: 0.5029 - loss: 3.7177 - mae: 0.4976
Epoch 1: val_loss improved from inf to 2.00972, saving model to best_model.h5
225/225 ━━━━━━━━━━━━━━━━━━━━ 156s 447ms/step - binary_accuracy: 0.5029 - loss: 3.7172 - mae: 0.4976 - val_binary_accuracy: 0.5300 - val_loss: 2.0097 - val_mae: 0.4959
Epoch 2/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step - binary_accuracy: 0.5010 - loss: 3.3092 - mae: 0.4988
Epoch 2: val_loss improved from 2.00972 to 1.97475, saving model to best_model.h5
225/225 ━━━━━━━━━━━━━━━━━━━━ 92s 411ms/step - binary_accuracy: 0.5010 - loss: 3.3087 - mae: 0.4988 - val_binary_accuracy: 0.5250 - val_loss: 1.9747 - val_mae: 0.4952
Epoch 3/100
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step - binary_accuracy: 0.5034 - loss: 3.0104 - mae: 0.4960
Epoch 3: val_loss improved from 1.97475 to 1.94346, saving model to best_model.h5
225/225 ━━━━━━━━━━━━━━━━━━━━ 91s 402ms/step - binary_accuracy: 0.5034 - loss: 3.0101 - mae: 0.4